# Compiling and Submitting Circuits onto Hilbert via Qiskit


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/hilbert/hilbert_compile_qss.ipynb) [![Launch Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/hilbert/hilbert_compile_qss.ipynb)

## Import Requirements

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the ```qiskit-superstaq``` client. You can try it out by running ```pip install qiskit-superstaq[examples]```:

In [1]:
# Required imports
try:
    import qiskit
    import qiskit_superstaq as qss
except ImportError:
    print("Installing qiskit-superstaq...")
    %pip install -q qiskit-superstaq[examples]
    print("Installed qiskit-superstaq. Please restart the kernel to import the packages.")

# Optional imports
import os  # Used if setting a token as an environment variable

To interface Superstaq via Qiskit, we must first instantiate a service provider in ```qiskit-superstaq```. We then supply a Superstaq API key (which you can get from https://superstaq.super.tech) by either providing the API key as an argument, i.e., ```qss.superstaq_provider.SuperstaqProvider(api_key="token")```, or by setting it as an environment variable (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_qss.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
provider = qss.superstaq_provider.SuperstaqProvider()

## Create a Circuit

First, we will create an example Qiskit circuit that we can then compile and submit onto Hilbert

In [3]:
circuit1 = qiskit.QuantumCircuit(1, 1)

circuit1.h(0)
circuit1.measure(0, 0)
circuit1.draw(fold=-1)

┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0

## Single Circuit Compilation

We will now compile the above circuit onto Hilbert's native gateset and visualize the differences by drawing the compiled circuit

In [4]:
compiler_output = provider.cq_compile(circuit1)

In [5]:
print(compiler_output.circuit)

      ┌─────────────────┐┌─┐
 q_0: ┤ GR(1.57, -1.57) ├┤M├
      └─────────────────┘└╥┘
 q_1: ────────────────────╫─
                          ║ 
 q_2: ────────────────────╫─
                          ║ 
 q_3: ────────────────────╫─
                          ║ 
 q_4: ────────────────────╫─
                          ║ 
 q_5: ────────────────────╫─
                          ║ 
 q_6: ────────────────────╫─
                          ║ 
 q_7: ────────────────────╫─
                          ║ 
 q_8: ────────────────────╫─
                          ║ 
 q_9: ────────────────────╫─
                          ║ 
q_10: ────────────────────╫─
                          ║ 
q_11: ────────────────────╫─
                          ║ 
q_12: ────────────────────╫─
                          ║ 
q_13: ────────────────────╫─
                          ║ 
q_14: ────────────────────╫─
                          ║ 
q_15: ────────────────────╫─
                          ║ 
q_16: ────────────────────╫─
              

If you would like to compile (or submit) on a different number of qubits, this can be done via the ```grid_shape``` option. This simply sets the shape of the rectangular qubit grid. However, specifying a grid that is incompatible with Hilbert's current capabilities will result in an error when submitting.

In [6]:
example_circuit = qiskit.QuantumCircuit(4, 1)
example_circuit.cx(0, 1)
example_circuit.cx(2, 3)
new_compiler_output = provider.cq_compile(example_circuit, grid_shape=(2, 2))
print(new_compiler_output.circuit.draw(fold=-1)

     ┌──────────────────┐     ┌─────────────────┐   ┌──────────────────┐     »
q_0: ┤0                 ├─────┤0                ├─■─┤0                 ├─────»
     │                  │┌───┐│                 │ │ │                  │┌───┐»
q_1: ┤1                 ├┤ Z ├┤1                ├─■─┤1                 ├┤ Z ├»
     │  GR(-0.79, 1.57) │└───┘│  GR(0.79, 1.57) │   │  GR(-0.79, 1.57) │└───┘»
q_2: ┤2                 ├─────┤2                ├─■─┤2                 ├─────»
     │                  │┌───┐│                 │ │ │                  │┌───┐»
q_3: ┤3                 ├┤ Z ├┤3                ├─■─┤3                 ├┤ Z ├»
     └──────────────────┘└───┘└─────────────────┘   └──────────────────┘└───┘»
«     ┌─────────────────┐
«q_0: ┤0                ├
«     │                 │
«q_1: ┤1                ├
«     │  GR(0.79, 1.57) │
«q_2: ┤2                ├
«     │                 │
«q_3: ┤3                ├
«     └─────────────────┘


## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in backend from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [7]:
backend = provider.get_backend("cq_hilbert_simulator")
job = backend.run(circuit1, shots=100)
print(job.result().get_counts())

{'0': 56, '1': 44}


## Multiple circuit compilation

All the functionalities we have seen so far can also be used on a multiple-circuit input as well. To illustrate this, let us create a different example two-qubit circuit, and compile both circuits we have created at the same time. 

In [8]:
circuit2 = qiskit.QuantumCircuit(2)
circuit2.cnot(0, 1)
circuit2.h(1)
circuit2.draw(fold=-1)

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

In [9]:
compiler_output = provider.cq_compile([circuit1, circuit2])

ValueError: Circuits failed to deserialize. This is likely because your version of qiskit-terra (0.24.1) is out of date. Consider updating it.

In [ ]:
print(compiler_output.circuits[0])
print()
print(compiler_output.circuits[1])